In [36]:
import pandas as pd

In [37]:
#Read in data into df from json
df = pd.read_json('../../src/response_iati.json')
df.iloc[385:388]

,iati_identifier,title_narrative,activity_date_type,activity_status_code,default_aid_type_code,description_narrative,last_updated_datetime,activity_date_iso_date,reporting_org_narrative,title_narrative_xml_lang,...,sector_narrative,document_link_url,sector_vocabulary,recipient_country_code,location_name_narrative,policy_marker_code,recipient_region_code,tag_code,tag_narrative,tag_vocabulary
385,44000-P131029,[Social Inclusion TAL],"[1, 2, 3, 4]",3,[C01],[The project development objective is to stren...,2024-01-28T00:53:37Z,"[2012-11-15T00:00:00Z, 2012-12-13T00:00:00Z, 2...",[World Bank],NaN,...,"[Public Administration - Social Protection, So...",[http://documents.worldbank.org/curated/en/172...,"[1, 1, 1, 1, 99, 99, 98, 98, 98, 98, 98, 98, 9...",[PE],"[Lambayeque, Tacna, Piura, Ucayali, Tumbes, Re...",NaN,NaN,NaN,NaN,NaN
386,44000-P132505,[Cusco Transport Improvement Project],"[1, 2, 3, 4]",2,[C01],[The overall PDO is to improve mobility in the...,2024-01-28T00:53:38Z,"[2013-09-19T00:00:00Z, 2014-02-28T00:00:00Z, 2...",[World Bank],NaN,...,"[Sub-National Government, Urban Transport, Pub...",[http://documents.worldbank.org/curated/en/578...,"[1, 1, 99, 99, 98, 98, 98, 98, 98]",[PE],[Provincia de Cusco],NaN,NaN,NaN,NaN,NaN
387,44000-P132515,[Support to the Subnational Transport Program ...,"[1, 2, 3, 4]",3,[C01],[The Project development objectives are to (i)...,2024-01-28T00:53:39Z,"[2013-12-12T00:00:00Z, 2015-12-11T00:00:00Z, 2...",[World Bank],NaN,...,"[Rural and Inter-Urban Roads, Public Administr...",[http://documents.worldbank.org/curated/en/787...,"[1, 1, 1, 1, 99, 99, 98, 98, 98, 98, 98, 98, 9...",[PE],"[Lambayeque, Region de San Martin, Region de H...",NaN,NaN,NaN,NaN,NaN


In [38]:
# create new empty df to fill with transformed data

trans_df = pd.DataFrame()

## **Feature Engineering & Transformation**

### IATI ID

In [39]:
df['iati_identifier'].describe()

count                      1000
unique                     1000
top       US-GOV-12-CO201491257
freq                          1
Name: iati_identifier, dtype: object

In [40]:
trans_df["iati_id"] = df["iati_identifier"].values

trans_df.head(2)

,iati_id
0,US-GOV-12-CO201491257
1,US-GOV-21-CO-579-48525


### Title

In [41]:
df['title_narrative'].describe()

count                                                  1000
unique                                                  646
top       [Protecting and Improving Health Globally:Buil...
freq                                                     19
Name: title_narrative, dtype: object

In [42]:
df["title_narrative_xml_lang"].describe()

count      967
unique       1
top       [en]
freq       967
Name: title_narrative_xml_lang, dtype: object

In [43]:
################
# Add en title #
################

trans_df["title_en"] = "NaN"
error_count = 0

for index, row in df.iterrows():
    lang_list = row['title_narrative_xml_lang']
    try:
        for j in range(0, len(lang_list)):
            if "en" in lang_list:
                if lang_list[j] == "en":
                    title = row['title_narrative'][j]
                    trans_df["title_en"][index] = title
            else:
                trans_df["title_en"][index] = "NaN"
            
    except:
        error_count += 1
    
print(f"Errors excepted: Probably no title for {error_count} sample(s)!")
print(trans_df["title_en"].describe())
print("Error excepted: Probably no title for one sample! ")
trans_df.head(2)

Errors excepted: Probably no title for 33 sample(s)!
count     1000
unique     614
top        NaN
freq        33
Name: title_en, dtype: object
Error excepted: Probably no title for one sample! 


,iati_id,title_en
0,US-GOV-12-CO201491257,Technical Assistance - GPI: Energy Procurement...
1,US-GOV-21-CO-579-48525,Collective memory and perspective of the post-...


In [44]:
###################
# Add other title #
###################

trans_df["title_other"] = "NaN"

error_count = 0

for index, row in df.iterrows():
    lang_list = row['title_narrative_xml_lang']
    try:
        for j in range(0, len(lang_list)):
            if lang_list[j] != "en":
                title = row['title_narrative'][j]
                if trans_df["title_other"][index] == "":
                    trans_df["title_other"][index] = title
                else:
                    trans_df["title_other"][index] = f"{trans_df['title_other'][index]}, {title}"
            
    except:
        error_count += 1
    
print(f"Errors excepted: Probably no title for {error_count} sample(s)!")
print(trans_df["title_other"].describe())
trans_df.head(2)

Errors excepted: Probably no title for 33 sample(s)!
count     1000
unique       1
top        NaN
freq      1000
Name: title_other, dtype: object


,iati_id,title_en,title_other
0,US-GOV-12-CO201491257,Technical Assistance - GPI: Energy Procurement...,NaN
1,US-GOV-21-CO-579-48525,Collective memory and perspective of the post-...,NaN
